# Platt SMO
- 简化版的SMO算法在大数据集上运行缓慢
- 两个版本的alpha更改和代数的优化环节一致，不同的在于选择alpha的方式
- 完整版的platt SMO应用了一些能提速的启发式方法

- 外循环选择第一个alpha值，并且选择过程在两种方式间交替
    - 一种是在数据集上扫描
    - 另一种是在非边界的alpha中实现单边扫描
    - 非边界指那些不等于边界0和C的alpha
- 选择第一个alpha值后，算法会通过一个内循环选择第二个alpha
    - 优化过程中会通过最大化步长的方式获得第二个alpha
    - 我们建立一个全局缓存用于保存误差值，并从中选择步长或者Ei-Ej最大的alpha


## 支持函数（Support Function）

In [1]:
import numpy as np
import random

# 加载数据
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat


# 调整alpha在H和L之间
def clipAlpha(aj,H,L):
    if aj>H:
        aj = H
    if aj<L:
        aj = L
    return aj

def selectJrand(i,m):
    # 随机选择j!=i
    j=i 
    while (j==i):
        j = int(random.uniform(0,m))
    return j

class optStruct():
    def __init__(self,dataMatIn,classLabels,C,toler,kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m,2)))

# 计算误差
def calcEk(oS,k):
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T * (oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 选择第二个alpha_j,希望能让alpha_j变化最大
def selectJ(i,oS,Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    
    oS.eCache[i] = [1,Ei]
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    
    if len(validEcacheList) >1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK,Ej
    else:
        j = selectJrand(i,oS.m)
        Ej = calcEk(oS,j)
    return j,Ej
    
    
def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]

## 完整的Platt SMO
### 内循环

In [2]:
# 内循环代码
def innerL(i,oS):
    Ei = calcEk(oS,i)
    # 1是不可容忍且还能调整  2是支持向量，且大于可容忍程度，还能调整
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) \
        or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i]>0)):
        j,Ej = selectJ(i,oS,Ei)
        
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        
        # 确定边界
        if oS.labelMat[i] != oS.labelMat[j]:
            k = oS.alphas[j] - oS.alphas[i]
            L = max(0,k)
            H = min(oS.C,oS.C+k)
        else:
            k = oS.alphas[j] + oS.alphas[i]
            L = max(0,k-oS.C)
            H = min(oS.C,k)          
        
        if L==H:
            return 0
        eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - \
            oS.X[j,:]*oS.X[j,:].T
        
        if eta>=0:
            return 0
        
        oS.alphas[j] -= oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)        
        updateEk(oS,j)
        
        if(abs(oS.alphas[j]-alphaJold)<0.00001):
            # print('j not moving enough')
            return 0
            
            
        oS.alphas[i] += oS.labelMat[i]*oS.labelMat[j]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T -\
                     oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T -\
                     oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T     
        
        if (0<oS.alphas[i]) and (oS.alphas[i]<oS.C):
            oS.b = b1
        elif (0<oS.alphas[j]) and (oS.alphas[j]<oS.C):
            oS.b = b2
        else:
            oS.b = (b1+b2)/2.0
        return 1
    else:
        return 0

### 外循环

In [3]:
# 外循环代码
def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):
    oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler,kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter<maxIter) and ((alphaPairsChanged >0)or entireSet):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
            print('fullSet, iter: %d i:%d, pairs changed %d'%(iter,i,alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
            print('non-bound, iter: %d i:%d, pairs changed %d'%(iter,i,alphaPairsChanged))
            iter += 1
            
        if entireSet:
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        
        print('iteration number: %d'% iter)
    return oS.b,oS.alphas

### 训练b,alpha

In [4]:
dataArr,labelArr = loadDataSet('../data/SVM/testSet.txt')

import time
t0 = time.clock()
b,alphas = smoP(dataArr,labelArr,0.6,0.001,40)
print(time.clock() - t0, "seconds process time")

fullSet, iter: 0 i:99, pairs changed 6
iteration number: 1
non-bound, iter: 1 i:54, pairs changed 0
iteration number: 2
fullSet, iter: 2 i:99, pairs changed 0
iteration number: 3
0.0909525 seconds process time


### 利用alpha计算W 

In [5]:
# 计算权重w
def calcWs(alphas,dataArr,classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).T
    m,n = X.shape
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [6]:
ws = calcWs(alphas,dataArr,labelArr)
ws

array([[ 0.73660705],
       [-0.33184906]])

### 测试

In [7]:
dataMat = np.mat(dataArr)

In [8]:
print('prediction is :',dataMat[0] * np.mat(ws) + b, ',   label is : ',labelArr[0])

prediction is : [[-1.72371307]] ,   label is :  -1.0


In [9]:
print('prediction is :',dataMat[1] * np.mat(ws) + b, ',   label is : ',labelArr[1])

prediction is : [[-2.30153899]] ,   label is :  -1.0


# 应用核函数处理非线性可分问题
- 核函数目的在于将原始低纬数据映射到高纬空间中，由核函数来实现这种映射
- 核函数有多种类型，经过空间转换后，我们可以在高纬空间中解决线性问题，这等价于在低维空间中解决非线性问题
- SVM优化中，存在向量内积运算，在此我们可以将内积运算换成核函数，这种技巧被称为**核技巧**

## 径向基核函数
- SVM中常用的一个核函数是径向基函数，基于向量距离运算输出一个标量
- 径向基函数的高斯版本具体如下，这里高斯函数将从特种空间映射到高纬（无穷）空间
$$
k(x,y) = \exp(\frac{-||x-y||^2}{2\sigma^2})
$$

In [10]:
def kernelTrans(X,A,kTup):
    m,n = np.shape(X)
    k = np.mat(np.zeros((m,1)))
    
    if kTup[0] == 'lin':
        k = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            k[j] = deltaRow *deltaRow.T
        k = np.exp(k/(-1*kTup[1]**2))
    else:
        print('The kernel is not recognized!')
    return k

In [11]:
class optStruct():
    def __init__(self,dataMatIn,classLabels,C,toler,kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m,2)))
        self.K = np.mat(np.zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X,self.X[i,:],kTup)

In [12]:
# 计算误差
def calcEk(oS,k):
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T * oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 内循环代码
def innerL(i,oS):
    Ei = calcEk(oS,i)
    # 1是不可容忍且还能调整  2是支持向量，且大于可容忍程度，还能调整
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) \
        or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i]>0)):
        j,Ej = selectJ(i,oS,Ei)
        
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        
        # 确定边界
        if oS.labelMat[i] != oS.labelMat[j]:
            k = oS.alphas[j] - oS.alphas[i]
            L = max(0,k)
            H = min(oS.C,oS.C+k)
        else:
            k = oS.alphas[j] + oS.alphas[i]
            L = max(0,k-oS.C)
            H = min(oS.C,k)          
        
        if L==H:
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        
        if eta>=0:
            return 0
        
        oS.alphas[j] -= oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)        
        updateEk(oS,j)
        
        if(abs(oS.alphas[j]-alphaJold)<0.00001):
            # print('j not moving enough')
            return 0
            
            
        oS.alphas[i] += oS.labelMat[i]*oS.labelMat[j]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] -\
                     oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j] -\
                     oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]    
        
        if (0<oS.alphas[i]) and (oS.alphas[i]<oS.C):
            oS.b = b1
        elif (0<oS.alphas[j]) and (oS.alphas[j]<oS.C):
            oS.b = b2
        else:
            oS.b = (b1+b2)/2.0
        return 1
    else:
        return 0

## 测试

In [13]:
def testRbf(k1 = 1.3):
    dataArr,labelArr = loadDataSet('../data/SVM/testSetRBF.txt')
    b,alphas = smoP(dataArr,labelArr,200,0.0001,10000,('rbf',k1))
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()

    # 构建支持向量
    svInd = np.nonzero(alphas.A>0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]

    print('there are %d Support Vectors'%sVs.shape[0])
    m,n = dataMat.shape

    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf',k1))
        predict = kernelEval.T*np.multiply(labelSV,alphas[svInd] )+ b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1

    print('the training error rate is : %f'%(float(errorCount)/m))

    # 测试测试集上的错误率
    dataArr,labelArr = loadDataSet('../data/SVM/testSetRBF2.txt')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    m,n = dataMat.shape

    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf',k1))
        predict = kernelEval.T*np.multiply(labelSV,alphas[svInd] )+ b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1

    print('the test error rate is : %f'%(float(errorCount)/m))
    
testRbf()

fullSet, iter: 0 i:99, pairs changed 25
iteration number: 1
non-bound, iter: 1 i:54, pairs changed 3
iteration number: 2
non-bound, iter: 2 i:54, pairs changed 0
iteration number: 3
fullSet, iter: 3 i:99, pairs changed 0
iteration number: 4
there are 21 Support Vectors
the training error rate is : 0.110000
the test error rate is : 0.210000


In [14]:
testRbf(0.1)

fullSet, iter: 0 i:99, pairs changed 85
iteration number: 1
non-bound, iter: 1 i:99, pairs changed 65
iteration number: 2
non-bound, iter: 2 i:99, pairs changed 1
iteration number: 3
non-bound, iter: 3 i:99, pairs changed 0
iteration number: 4
fullSet, iter: 4 i:99, pairs changed 6
iteration number: 5
non-bound, iter: 5 i:99, pairs changed 0
iteration number: 6
fullSet, iter: 6 i:99, pairs changed 0
iteration number: 7
there are 88 Support Vectors
the training error rate is : 0.000000
the test error rate is : 0.080000


- sigma影响径向基函数，降低simga往往会获得更多的支持向量。
- 支持向量太少，会得到相对较差的决策边界
- 支持向量太多，那么每次都利用太多数据进行分类，这就等价于k近邻

# 手写数字识别
之前使用KNN构建手写数字识别，但是其每次要保留数据集，占用的内存太大。如何保持性能不变而使用更少的内存呢？对此可以考虑使用SVM，保留的样本少了很多，但是能获得接近的效果

In [15]:
def img2vector(filename):
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect    

In [16]:
from os import listdir

def loadImages(dirName):
    hwlabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = np.zeros((m,1024))
    for i in range(m):
        # 获取标签
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwlabels.append(-1)
        # 获取数据
        else:
            hwlabels.append(1)
        trainingMat[i,:] = img2vector(dirName + '/' + fileNameStr)
    return trainingMat,hwlabels


In [23]:
def testDigits(kTup = ('rbf',10)):
    dataArr,labelArr = loadImages('../data/SVM/hw_data/trainingDigits')
    b,alphas = smoP(dataArr,labelArr,200,0.0001,10000,kTup)

    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T

    # 构建支持向量
    svInd = np.nonzero(alphas.A>0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print('there are %d Support Vectors'%np.shape(sVs)[0])

    m,n = dataMat.shape
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict = kernelEval.T*np.multiply(labelSV,alphas[svInd] )+ b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print('the training error rate is : %f'%(float(errorCount)/m))


    # 测试测试集上的错误率
    dataArr,labelArr = loadImages('../data/SVM/hw_data/testDigits')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    m,n = dataMat.shape

    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict = kernelEval.T*np.multiply(labelSV,alphas[svInd] )+ b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1

    print('the test error rate is : %f'%(float(errorCount)/m))


In [24]:
testDigits(('rbf',10))

fullSet, iter: 0 i:401, pairs changed 98
iteration number: 1
non-bound, iter: 1 i:317, pairs changed 15
iteration number: 2
non-bound, iter: 2 i:362, pairs changed 0
iteration number: 3
fullSet, iter: 3 i:401, pairs changed 0
iteration number: 4
there are 121 Support Vectors
the training error rate is : 0.000000
the test error rate is : 0.010000


In [25]:
testDigits(('rbf',5))

fullSet, iter: 0 i:401, pairs changed 401
iteration number: 1
non-bound, iter: 1 i:401, pairs changed 401
iteration number: 2
non-bound, iter: 2 i:401, pairs changed 391
iteration number: 3
non-bound, iter: 3 i:401, pairs changed 369
iteration number: 4
non-bound, iter: 4 i:401, pairs changed 214
iteration number: 5
non-bound, iter: 5 i:401, pairs changed 26
iteration number: 6
non-bound, iter: 6 i:401, pairs changed 0
iteration number: 7
fullSet, iter: 7 i:401, pairs changed 0
iteration number: 8
there are 402 Support Vectors
the training error rate is : 0.000000
the test error rate is : 0.060000


- sigma取10的时候效果通常不错
- 另外线性核函数的效果并不一定很差，可以考虑牺牲线性核函数错误率来换取分类速度

# 总结
- 支持向量机泛化错误率低，有很好的学习能力，且有很好的推广性，因而使得svm十分流行
- 有人认为svm是最好的定式算法
- 过去试图求解二次优化问题来最大化分类间隔，训练支持向量机非常复杂且低效；John Platt引入SMO算法，每次优化两个alpha来加速svm训练
- 核方法从低维空间将数据映射到高纬空间，因此在低维空间中非线性问题可以在转换为高纬空间中的线性问题来求解
- 核方法不止可用于svm还可以用于其他算法，其中径向基函数是一个常用的度量两个向量距离的核函数
- 支持向量机是一个二分类器，解决多分类问题时需要进行扩展
- SVM效果也对优化参数和核函数中的参数很敏感
